In [2]:
import pandas as pd
import numpy as np
import math
import random


ignore_features = ['25', '26', '8', '58', '20', '23', '29', '31', '32', '35']
toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[1] != "numeric" and line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)


In [3]:
data_end = np.shape(data_df)[0]
df = data_df.append(quiz_df)
df_trans = pd.get_dummies(df, columns=toencode.keys(), dummy_na=False)
data = np.array(df_trans)

data, quiz = data[:data_end], data[data_end:]

In [4]:
#toencode
#print data_df

In [5]:
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [6]:
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error

In [6]:
from sklearn.neighbors import KNeighborsClassifier
lab = np.ravel(lab)
neigh = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
f = neigh.fit(train, lab) 

In [ ]:
#print np.shape(train)
#print np.shape(test)
preds = neigh.predict(test)

In [64]:
#x = []
#for i in xrange(len(preds)):
#    p = preds[i]
#    x.append((i+1, p))

#preds_df = pd.DataFrame(x, columns=['Id', 'Prediction'])
#preds_df.to_csv('knn_out.csv', index=False)

from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(train,lab)

In [9]:
testlabelnum = [value[0] for value in testlabel]
labnum = [value[0] for value in testlabel]

In [41]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
pca.fit(train,lab)

PCA(copy=True, n_components=300, whiten=False)

In [42]:
traintrans = pca.fit_transform(train,lab)
np.shape(traintrans)

(25367, 300)

In [43]:
testtrans = pca.transform(test)
np.shape(testtrans)

(101470, 300)

In [44]:
ylogregtesttrans = logreg.fit(traintrans, lab).predict(testtrans)

/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:
eval(ylogregtesttrans,testlabelnum)

0.8772839262836306

In [46]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=600,learning_rate=1.5,
    algorithm="SAMME")

yadaboosttest = adaboost.fit(train, lab).predict(test)
print eval(yadaboosttest, testlabelnum)

0.888351236819


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
